In [16]:
import requests
import time
import pandas as pd
from joblib import Parallel, delayed
from tqdm import tqdm
from tqdm_joblib import tqdm_joblib

/Users/nicross/opt/anaconda3/envs/py310/lib/python3.10/site-packages/tqdm_joblib/__init__.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [11]:
url = 'https://api.steampowered.com/ISteamApps/GetAppList/v2/'
params = {
    'key': '3B74BFF7710FBDFABE2684E5ECE043D7',
    'format': 'json'
}
resp = requests.get(url, params=params)
resp.raise_for_status()

apps = resp.json()['applist']['apps']

df = pd.DataFrame(apps)[['appid', 'name']]

In [18]:
STORE_URL = "https://store.steampowered.com/api/appdetails"

def fetch_tags(appid):
    """Fetch genres and subgenres (categories) for a single AppID."""
    try:
        resp = requests.get(
            STORE_URL,
            params={"appids": appid, "cc": "us", "l": "en"},
            timeout=10
        )
        resp.raise_for_status()
        data = resp.json().get(str(appid), {})
        if not data.get("success"):
            return [], []
        info = data["data"]
        genres  = [g["description"] for g in info.get("genres", [])]
        subcats = [c["description"] for c in info.get("categories", [])]
    except Exception:
        genres, subcats = [], []
    time.sleep(0.2)
    return genres, subcats

appids = df["appid"].tolist()

# wrap the Parallel call so tqdm_joblib shows a single progress bar
with tqdm_joblib(tqdm(desc="Fetching tags", total=len(appids), leave=False)) as progress:
    results = Parallel(
        n_jobs=8,
        backend="threading"
    )(delayed(fetch_tags)(appid) for appid in appids)

# unzip back into the DataFrame
df["genres"], df["subgenres"] = zip(*results)


ching tags:   0%|                                 | 0/244588 [00:00<?, ?it/s]

Fetching tags:   0%|                                 | 0/244588 [02:06<?, ?it/s]


                                    | 1/244588 [00:00<25:23:43,  2.68it/s]

                                     | 9/244588 [00:00<4:27:55, 15.21it/s]

                                    | 17/244588 [00:00<3:21:36, 20.22it/s]

                                    | 25/244588 [00:01<3:04:23, 22.11it/s]

                                    | 33/244588 [00:01<2:57:10, 23.01it/s]

                                    | 41/244588 [00:01<2:50:58, 23.84it/s]

                                    | 48/244588 [00:02<2:19:38, 29.19it/s]

                                    | 52/244588 [00:02<2:43:59, 24.85it/s]

                                    | 57/244588 [00:02<2:58:23, 22.85it/s]

                                    | 64/244588 [00:02<2:16:57, 29.76it/s]

                                    | 68/244588 [00:02<2:50:09, 23.95it/s]

 

KeyboardInterrupt: 